Games and gambling have been part of human cultures around the world.  Nowadays, the connection between games of chance and mathematics (in particular probability) are so obvious that it is taught to school children.  However, the mathematics of games and gambling only started to formally [develop](https://en.wikipedia.org/wiki/Gambling#History) in the 17th century with the works of multiple mathematicians such as Fermat and Pascal.  It is then no wonder that many incorrect beliefs around gambling have formed that are "intuitive" from layman's perspective but fail to pass muster when applying the rigor of mathematics.

In this post, I want to discuss how surprisingly easy it is to be fooled into the wrong line of thinking even when approaching it using mathematics.  We'll take a look from both a theoretical (mathematics) point of view looking at topics such as the [Gambler's Fallacy](https://en.wikipedia.org/wiki/Gambler's_fallacy) and the [law of small numbers](https://en.wikipedia.org/wiki/Hasty_generalization) as well as do some simulations using code to gain some insight into the problem.  This post was inspired by a paper I recently came across a paper by Miller and Sanjurjo[<sup>[1]</sup>](#fn-1) that explains the suprising result of how easily we can be fooled.

<!-- TEASER_END -->

## The Laws of Large (and Small) Numbers

Let's start by taking a look at one of the simplest situations we can think of: flipping a fair coin.  By "fair", we usually mean that it has a 50% chance of leanding heads (or "H" for short) and 50% change of landing tail (or "T").  More formally:

$$P(H) = 0.5$$
$$P(T) = 0.5$$

What about flipping a fair coin N times?  We expect to get roughly half of the coins to end up H and half T. This is confirmed by [Borel's law of large numbers](https://en.wikipedia.org/wiki/Law_of_large_numbers#Borel.27s_law_of_large_numbers) (one of the various forms) that states:

> If an experiment is repeated a large number of times, independently under identical conditions, then the proportion of times that any specified event  occurs approximately equals the probability of the event's occurrence on any particular trial; the larger the number of repetitions, the better the approximation tends to be.

Let's see exactly how man repetitions we need to get close.  Let's first define some code to do our fair coin flip and also simulations of the fair coin flip.

In [84]:
import random 

def flip():
    return random.choice(['H', 'T'])

def simulate(N):
    return [1 if flip() == 'H' else 0 for x in range(N)]

def flip_ratio(flips):
    N = len(flips)
    heads = sum(flips)
    tails = N - heads
    # Python 3: implicit float division, lovely.
    return (heads / N, tails / N)

Next, let's simulate for several \\(N\\) to see how many repetitions is enough.

In [423]:
from pandas import Series, DataFrame

def pretty(result):
    return '(%.3f, %.3f)' % (result[0], result[1])

def toseries(results):
    return Series(results, index=['Run %d' % i for i in range(1, len(results) + 1)])

nvalues = [10, 50, 100, 1000, 10000]
runs = 4
data = [pretty(flip_ratio(simulate(10))) for x in range(runs)]

# Put it in pandas data frame for pretty printing
DataFrame([toseries(data) for n in nvalues], index=['N=%d' % n for n in nvalues])

,Run 1,Run 2,Run 3,Run 4
N=10,"(0.500, 0.500)","(0.400, 0.600)","(0.400, 0.600)","(0.300, 0.700)"
N=50,"(0.500, 0.500)","(0.400, 0.600)","(0.400, 0.600)","(0.300, 0.700)"
N=100,"(0.500, 0.500)","(0.400, 0.600)","(0.400, 0.600)","(0.300, 0.700)"
N=1000,"(0.500, 0.500)","(0.400, 0.600)","(0.400, 0.600)","(0.300, 0.700)"
N=10000,"(0.500, 0.500)","(0.400, 0.600)","(0.400, 0.600)","(0.300, 0.700)"


When \\(N=10\\), we can get some pretty wild swings like in Run 4 with only 30% of the flips resulting in heads.  Even at \\(N=50\\), we're still not quite close to the 50/50 mark.  Once we start gettting past \\(N=100\\), we start getting quite close to a 50/50 distribution with tighter range of values around \\(0.5\\).

This leads to a very important point when trying to reason with the law of large numbers:

> Random events only converge to their average in the **long-run**.

The corollary to this rule is:

> In the short-run **anything** can happen.

If you've ever been in a casino, the last statement will ring true (for better or worse).  Not recognizing this second rule is essentially the [law of small numbers](https://en.wikipedia.org/wiki/Hasty_generalization)[<sup>[2]</sup>](#fn-2) (also known as hasty generalization/induction): 

> [The law of small numbers] is an informal fallacy of faulty generalization by reaching an inductive generalization based on insufficient evidence—essentially making a hasty conclusion without considering all of the variables. In statistics, it may involve basing broad conclusions regarding the statistics of a survey from a small sample group that fails to sufficiently represent an entire population.

For example on the \\(N=10\\) run (or even \\(N=50\\)), we might have erroneously concluded that our coin was not fair because of the big deviation between heads and tails.

## Independent Events and Gambler's Fallacy

Now let's take a look at another concept about random events: [independence](https://en.wikipedia.org/wiki/Independence_%28probability_theory%29).  The definition is basically what you intuitively think it might be:

> The occurrence of one [event] does not affect the probability of the other.

Going back to our fair coin flipping example, each toss of our coin is independent from the other.  Therefore each coin toss, regardless of what has happened before, has a 50/50 chance of being heads or tails.  Easy to think about abstractly but what if we got a sequence of coin flips like this:

    H H H H T H H H H ?

What would you expect the next flip to be?  It's tempting to think it has a slighly higher change of being T but because of independence, it still has a 50/50 chance of being heads or tails.  This almost natural tendency to believe that T should come up next (and ignore the independence of the events) is called the [Gambler's Fallacy](https://en.wikipedia.org/wiki/Gambler's_fallacy):

> The gambler's fallacy, also known as the Monte Carlo fallacy or the fallacy of the maturity of chances, is the mistaken belief that, if something happens more frequently than normal during some period, it will happen less frequently in the future, or that, if something happens less frequently than normal during some period, it will happen more frequently in the future (presumably as a means of balancing nature). 

You might think that this fallacy is so obvious that no one would make this mistake, you would be wrong.  You don't have to look any further than your local casino where you can see every roulette wheel electronic display showing the last ten or so spins [<sup>[3]</sup>](#fn-3).  Many casino patrons will use this screen to religiously count how many red and black numbers have come up, along with a bunch of other various statistics in hopes that they might predict the next spin.  Of course each spin in independent, so these statistics won't help at all but that doesn't stop the casino into letting people throw their money away.

## An Unexpected Simulation

Now that we have an understanding of the law of large numbers, independent events and the gambler's fallacy, let's try to simulate a situation where we might run into the gambler's fallacy.

Let's concoct a situation.  Take our fair coin.  Flip it \\(4\\) times in a row, write down each outcome and on a piece of paper (or equivalently save it in memory).  Next, count the number of outcomes that immediately followed a heads, and the number of those outcomes that were heads.  Essentially, we're trying to compute \\(P(H|H)\\).  Now repeat that experiment \\(N\\) times.  We would expect that the chance of getting a H is roughly \\(0.5\\), or \\(P(H|H) = 0.5\\).  Let's see if our intuition matches the emperical results.

First, we can reuse our `simulate()` function from before to flip the coin 4 times.  Next, we'll add two more functions to help us count the outcomes that immediately follow heads (and whether or not it is a heads), as well as running the \\(N\\) experiments.

In [432]:
def count_heads_after_heads(flips):
    heads_after_heads = 0
    total = 0
    # Need at least one flip
    for i in range(1, len(flips)):
        if flips[i-1] == 1:
            total += 1
            if flips[i] == 1:
                heads_after_heads += 1
    
    return (heads_after_heads, total)

def simulate_fallacy(num_flips, N):
    p = []
    for x in range(N):
        r = count_heads_after_heads(simulate(num_flips))
        if r[1] > 0:
            p += [r[0] / r[1]]
    
    return p

Now let's run through various values of \\(N\\), again we'll do several runs at each N to get an idea of how the experiment varies.

In [442]:
def gen_fallacy_data(runs, num_flips, Ns):
    data = []
    for N in Ns:
        sdata = []
        for run in range(runs):
            p = simulate_fallacy(num_flips, N)
            avg = sum(p) / len(p)
            sdata += ['%.4f' % avg]
        data += [toseries(sdata)]
    return data
    
runs = 4
num_flips = 4
Ns = [10, 100, 1000, 10000, 100000]
DataFrame(gen_fallacy_data(runs, num_flips, Ns), index=['N=%d' % n for n in Ns])

,Run 1,Run 2,Run 3,Run 4
N=10,0.4167,0.4630,0.3889,0.1667
N=100,0.4008,0.4314,0.3841,0.3294
N=1000,0.4270,0.3961,0.4280,0.4046
N=10000,0.4106,0.3983,0.4046,0.4131
N=100000,0.4052,0.4049,0.4038,0.4056


Suprised by the results?  There's definitely something fishy going on here.  Even for very large \\(N\\), we're getting values which seem to converge on \\(0.4\\).  At \\(N=100000\\), we should definitely start getting into the territory of the law of large numbers.

Let's try to play around with the parameters.  How about increasing `num_flips`?

In [443]:
runs = 4
num_flips = 10
Ns = [10, 100, 1000, 10000]
DataFrame(gen_fallacy_data(runs, num_flips, Ns), index=['N=%d' % n for n in Ns])

,Run 1,Run 2,Run 3,Run 4
N=10,0.3958,0.2917,0.4367,0.4148
N=100,0.4419,0.4179,0.4248,0.4520
N=1000,0.4351,0.4445,0.4375,0.4418
N=10000,0.4453,0.4461,0.4402,0.4445


Interesting, it seems to be converging to a different number now.  Let's keep pumping it up and see what happens.

In [449]:
from IPython.display import display

runs = 4
Ns = [10, 100, 1000]
for nf in [100, 1000, 5000]:
    df = DataFrame(gen_fallacy_data(runs, nf, Ns), 
                   index=['N=%d' % n for n in Ns])
    print("Number of flips = %d:" % nf)
    display(df)

Number of flips = 100:


,Run 1,Run 2,Run 3,Run 4
N=10,0.5095,0.5142,0.4685,0.5295
N=100,0.4948,0.5140,0.4939,0.4863
N=1000,0.4966,0.4949,0.4922,0.4926


Number of flips = 1000:


,Run 1,Run 2,Run 3,Run 4
N=10,0.5031,0.5005,0.5001,0.5033
N=100,0.5009,0.4995,0.4971,0.4987
N=1000,0.4986,0.5014,0.5002,0.5000


Number of flips = 5000:


,Run 1,Run 2,Run 3,Run 4
N=10,0.4958,0.4975,0.4967,0.5017
N=100,0.4987,0.4997,0.5003,0.4998
N=1000,0.4996,0.4995,0.4998,0.4999


## Notes

List of Notes: [^1], [^2], [^3]

[^1]: Miller, Joshua Benjamin and Sanjurjo, Adam, Surprised by the Gambler's and Hot Hand Fallacies? A Truth in the Law of Small Numbers (September 15, 2015). IGIER Working Paper #552. Available at SSRN: http://ssrn.com/abstract=2627354 or http://dx.doi.org/10.2139/ssrn.2627354 

[^2]: Of course it's not *really* a law, especially since it is a fallacy.

[^3]: Check out this story: [Las Vegas Roulette Wheel Stops on Same Number 7 Times in a Row](http://gawker.com/5919675/what-are-the-odds-las-vegas-roulette-wheel-stops-on-same-number-seven-times-in-a-row).  Imagine you were there when the wheel stopped on the same number for the sixth time.  How tempted would you be to make a huge bet on it *not* coming up to that number on the seventh time?
